In [1]:
import torch
from torch import nn

In [2]:
import torch.nn.functional as F

In [7]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
# class MLP(nn.Module):
#   def __init__(self,activation):
#     super().__init__()
#     self.net = nn.Sequential(
#         nn.Flatten(),
#         nn.Linear(28*28,256),
#         activation(),
#         nn.Linear(256,64),
#         activation(),
#         nn.Linear(64,10)
#     )

# def forward(self,x):
#   return self.net(x)

In [4]:
class MLP(nn.Module):
    def __init__(self, activation_fn):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.activation = activation_fn

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        return self.fc3(x)


In [5]:
class GhostReLU(nn.Module):
  def forward(self,x):
    return x*torch.sigmoid(1.735*torch.tanh(x))

In [10]:
class Pulsar(nn.Module):
    def __init__(self, alpha_init=0.01, pulsar_push=0.01):
        super().__init__()
        self.alpha = nn.Parameter(torch.tensor(alpha_init))  # Trainable slope
        self.pulsar_push = pulsar_push
        self.register_buffer('x_prev', None)

    def forward(self, x):
        if self.x_prev is None or self.x_prev.shape != x.shape:
            self.x_prev = torch.zeros_like(x)

        direction = torch.sign(x - self.x_prev)
        at_zero = (x == 0)
        negative = (x < 0)

        # Classic PReLU behavior for non-zero values
        prelu_out = torch.where(negative, self.alpha * x, x)

        # Inject Pulsar push ONLY at zero
        final_out = torch.where(at_zero, self.pulsar_push * direction, prelu_out)

        self.x_prev = x.detach()
        return final_out


In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1000)


100%|██████████| 9.91M/9.91M [00:11<00:00, 893kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 131kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.24MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.68MB/s]


In [9]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
    return correct / len(test_loader.dataset)


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Choose one: GhostReLU or PulsarHybrid()
# model = MLP(GhostReLU()).to(device)
model = MLP(Pulsar()).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, criterion, epoch)
    acc = test(model, device, test_loader)
    print(f"Epoch {epoch} — Test Accuracy: {acc*100:.2f}%")


Epoch 1 — Test Accuracy: 96.57%
Epoch 2 — Test Accuracy: 96.82%
Epoch 3 — Test Accuracy: 97.70%
Epoch 4 — Test Accuracy: 97.43%
Epoch 5 — Test Accuracy: 97.55%


In [12]:
model = MLP(GhostReLU()).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, criterion, epoch)
    acc = test(model, device, test_loader)
    print(f"Epoch {epoch} — Test Accuracy: {acc*100:.2f}%")

Epoch 1 — Test Accuracy: 95.95%
Epoch 2 — Test Accuracy: 96.61%
Epoch 3 — Test Accuracy: 97.34%
Epoch 4 — Test Accuracy: 97.69%
Epoch 5 — Test Accuracy: 97.51%


In [13]:
model = MLP(nn.ReLU()).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, criterion, epoch)
    acc = test(model, device, test_loader)
    print(f"Epoch {epoch} — Test Accuracy: {acc*100:.2f}%")

Epoch 1 — Test Accuracy: 97.02%
Epoch 2 — Test Accuracy: 96.94%
Epoch 3 — Test Accuracy: 97.34%
Epoch 4 — Test Accuracy: 97.71%
Epoch 5 — Test Accuracy: 97.80%


In [14]:
model = MLP(nn.LeakyReLU(negative_slope = 0.01)).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, criterion, epoch)
    acc = test(model, device, test_loader)
    print(f"Epoch {epoch} — Test Accuracy: {acc*100:.2f}%")

Epoch 1 — Test Accuracy: 95.95%
Epoch 2 — Test Accuracy: 97.41%
Epoch 3 — Test Accuracy: 97.78%
Epoch 4 — Test Accuracy: 96.98%
Epoch 5 — Test Accuracy: 97.96%


In [15]:
model = MLP(nn.PReLU(num_parameters=1)).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, criterion, epoch)
    acc = test(model, device, test_loader)
    print(f"Epoch {epoch} — Test Accuracy: {acc*100:.2f}%")

Epoch 1 — Test Accuracy: 96.06%
Epoch 2 — Test Accuracy: 97.54%
Epoch 3 — Test Accuracy: 97.68%
Epoch 4 — Test Accuracy: 97.54%
Epoch 5 — Test Accuracy: 97.43%
